In [37]:
from random import randint, seed


def shortestVector(matrix):
    """
    Returns the norm of the shortest vector and its index.
    """
    return sorted([(matrix[idx].norm().n(), matrix[idx], idx) for idx in range(matrix.nrows())])[0]

def randomMatrix(dimension, per) -> matrix:
    list = [randint(-per, per) for _ in range(dimension**2)]
    M = matrix(ZZ, dimension, dimension, list)
    while M.rank() != dimension:
        list = [randint(-per, per) for _ in range(dimension**2)]
        M = matrix(ZZ, dimension, dimension, list)
    return M

def gram_matrix(matrix) -> matrix:
    """
    Calculate Gram matrix of given matrix.
    :param matrix: given matrix
    :return: Gram matrix of matrix
    """
    return matrix * matrix.transpose()


def find_real_minimum(G, i, x) -> vector:
    matrixA = matrix(G.nrows() - 1, G.nrows() - 1, 0)
    matrixB = matrix(G.nrows() - 1, 1, 0)
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(G.nrows()):
        if row != i:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != i]
            matrixB[b] = sum([x * G[row,j] for j in range(len(G[row])) if j == i])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right((-1)*matrixB)).list()
    result.insert(i,x)
    return vector(result).n(digits=5) 



In [49]:
def innerComparsion(dimension, perimeter, B, B_red, v_min, x, G,i) -> dict:
    solutions = find_real_minimum(G, i, x[i])
    for j in range(dimension - 1):  # zbytecny forcyklus???
        difference = x[j] - solutions[j]
        if difference > 1.5: 
            if all([sol == 0 for sol in solutions]):
                break  
            return into_dict(B, G, x, solutions)
        
def matrix_to_list(A) -> list:
    return [[int(A.list()[row * A.ncols() + col]) for col in range(A.ncols())] for row in range(A.nrows())]

def vector_to_list(vector) -> list:
    return [float(num) for num in vector.list()]


def into_dict(B, G, x, solutions) -> dict:
    result = {}
    result["B"] = B
    result["G"] = G
    result["The linear combination given by LLL"] = x
    result["The linear combination suggested by cube"] = solutions
    result["The difference"] = x - vector(solutions)
    result["Shortest vector given by LLL"] = x*B
    return result


In [61]:
import json

def main():
    output_data = []
    for iteration in range(1000):
        dimension = 4
        perimeter = 50
        B = randomMatrix(dimension, perimeter)
        B_red = B.LLL()
        v_min = shortestVector(B_red)[1]
        x = B.solve_left(v_min) # find linear combination corresponding to shortest vector found by LLL
        G = gram_matrix(B)
        for i in range(G.nrows()):
            data = innerComparsion(dimension, perimeter, B, B_red, v_min, x, G,i) # terrible
            if data != None:
                output_data.append(data)
    print((output_data))
#     print_the_output(output_data)
    into_json(format_data(output_data))

def format_data(output_data):
    for dic in output_data:
        for key, value in dic.items():
            if isinstance(value, sage.matrix.matrix_integer_dense.Matrix_integer_dense):
                dic[key] = matrix_to_list(value)
            else: # here I assume that output_data consits only f matrices and one-line entries
                dic[key] = vector_to_list(value)
    return output_data

def into_json(output_data):
    out_file = open("myfile.json", "w") 
    json.dump(output_data, out_file) 
    out_file.close() 


def print_the_output(output_data) -> None:
    for dict in output_data:
        for pair in dict.items():
            print(pair[1], ": ", pair[0])
        print()
    
main()
print("Done")

[{'B': [  5 -45  23  11]
[-12 -45 -47  21]
[  7 -40 -37  -8]
[ 44 -16 -50 -50], 'G': [2700 1115  896 -760]
[1115 4819 3287 1492]
[ 896 3287 3082 3198]
[-760 1492 3198 7192], 'The linear combination given by LLL': (1, 1, -2, 1), 'The linear combination suggested by cube': (1.0000, 1.8667, -3.6937, 1.3608), 'The difference': (0.00000, -0.86671, 1.6937, -0.36084), 'Shortest vector given by LLL': (23, -26, 0, -2)}, {'B': [-38  42  19 -27]
[ -9   6   1  -7]
[-41  30  45 -12]
[-27  -7  44   1], 'G': [4298  802 3997 1541]
[ 802  167  678  238]
[3997  678 4750 2865]
[1541  238 2865 2715], 'The linear combination given by LLL': (3, -5, -3, 2), 'The linear combination suggested by cube': (3.5705, -6.9011, -3.2258, 2.0000), 'The difference': (-0.57052, 1.9011, 0.22577, 0.00000), 'Shortest vector given by LLL': (0, -8, 5, -8)}, {'B': [  2  33  33  33]
[  5  26 -41 -45]
[ 19  39  11  15]
[ 15  34 -38 -17], 'G': [ 3271 -1970  2183  -663]
[-1970  4407   -17  3282]
[ 2183   -17  2228   938]
[ -663  32

In [44]:
# for single matrix

H = matrix([[5,2,4],[1,3,5],[4,5,2]])
HLLL = H.LLL()
v_min = shortestVector(HLLL)[1]
h = H.solve_left(v_min) # find linear combination corresponding to shortest vector found by LLL
for i in range(H.nrows()):
    variables = [var('y_{}'.format(j)) for j in range(gram_matrix(H).nrows())]
    y = deepcopy(variables)
    y[i] = h[i]
    Y =  matrix(1, H.nrows(), y)
    to_minimize = (Y * gram_matrix(H) * Y.transpose())[0]
    print(to_minimize)

((35*y_1 + 29*y_2 - 31)*y_1 + (29*y_1 + 45*y_2 - 38)*y_2 - 31*y_1 - 38*y_2 + 45)
((45*y_0 + 38*y_2)*y_0 + (38*y_0 + 45*y_2)*y_2)
((45*y_0 + 31*y_1 + 38)*y_0 + (31*y_0 + 35*y_1 + 29)*y_1 + 38*y_0 + 29*y_1 + 45)


In [53]:
H.list()
matrix_to_list(H)
type(H)
type(H) is sage.matrix.Matrix



AttributeError: module 'sage.matrix' has no attribute 'Matrix'

In [54]:
type(H)

<class 'sage.matrix.matrix_integer_dense.Matrix_integer_dense'>